In [87]:
import sickle
from tqdm import tqdm

In [97]:
tlist = ['cardiovascular', 'cardiogram', 'cardiology', 'heart', 'vascular']
base_url = 'http://dspace.library.uu.nl/oai/dissertation'
pdf_path = '//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PhDTheses'

In [55]:
sickler = sickle.Sickle(base_url)


In [56]:
sets = sickler.ListSets()

In [57]:
Sets = {}
for s in sets:
    Sets[s.setSpec]  = s.setName    

In [59]:
Sets_to_mine = []
for key, val in Sets.items():
    if 'umc' in val.lower():
        print(f'Set: {key} contains umcu')
        Sets_to_mine.append(key)
    elif 'diss' in val.lower():
        print(f'Set: {key} contains diss')
        Sets_to_mine.append(key)
    elif 'phd' in val.lower():
        print(f'Set: {key} contains phd') 
        Sets_to_mine.append(key)
    

Set: dissertation contains diss
Set: com_1874_298213 contains umcu
Set: col_1874_298214 contains umcu


In [100]:
link_list = []
error_list = []
for set_to_mine in tqdm(Sets_to_mine):
    records = sickler.ListRecords(metadataPrefix='oai_dc', ignore_deleted=True, set=set_to_mine) # dissertation com_1874_298213

    relevant_counter = 0
    for r in tqdm(records):
        meta = r.get_metadata()
        relevant = False
        
        try:
            if any([t in subj for subj in meta['subject'] for t in tlist]):
                relevant = True
        except:
            pass
            
        try:
            if any([t in subj for subj in meta['description'] for t in tlist]):
                relevant = True
        except:
            pass          
        
        if relevant:
            relevant_counter += 1
            link = meta['identifier'][0]
            linkPdf = link.replace('dspace.library.uu.nl/', 'dspace.library.uu.nl/bitstream/')
            linkPdf = linkPdf + '/full.pdf'
            try:
                link_list.append({'Set':set_to_mine, 
                                'Link': link, 
                                'PdfLink': linkPdf,
                                'Title': meta['title'][0],
                                'Description': meta['description'][0],
                                'Date': meta['date'][0],
                                'Language': meta['language'][0],
                                }
                                )
            except Exception as e:
                link_list.append({'Set':set_to_mine, 
                                'Link': link, 
                                'PdfLink': linkPdf,
                                'Title': "whoopsie",
                                'Description': "daisey",
                                'Date': "in the present",
                                'Language': "klingon?",
                                }
                                )
                error_list.append(f'Error: {e} for link: {link}')
    print(f'Found {relevant_counter} relevant records in set: {set_to_mine}')      

10398it [01:52, 92.58it/s]0<?, ?it/s]
 33%|███▎      | 1/3 [01:52<03:44, 112.38s/it]

Found 993 relevant records in set: dissertation


3463it [00:28, 122.49it/s]
 67%|██████▋   | 2/3 [02:20<01:02, 62.93s/it] 

Found 796 relevant records in set: com_1874_298213


3463it [00:28, 123.23it/s]
100%|██████████| 3/3 [02:49<00:00, 56.34s/it]

Found 796 relevant records in set: col_1874_298214


In [101]:
# now we parse the link list and download the pdfs
import requests
import os

# add sleep
from time import sleep

pdf_error_list = []
for link in tqdm(link_list):
    pdf_url = link['PdfLink']
    pdf_name = link['Link'].split('/')[-1] + '.pdf'
    _pdf_path = os.path.join(pdf_path, pdf_name)
    
    # try to download the pdf
    try:
        r = requests.get(pdf_url, stream=True)
        with open(_pdf_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
    except Exception as e:
        pdf_error_list.append(f'Error: {e} for link: {link["Link"]}')
        
    sleep(1)

  5%|▌         | 138/2585 [04:43<1:11:30,  1.75s/it]